# Day Count Convention Analysis

This notebook demonstrates the day count convention mismatch that creates
the arbitrage opportunity in Canadian Government bonds.

## Key Concepts
1. **Actual/Actual**: Used for quoted bond prices
2. **Actual/365**: Used for settlement cash flows
3. **Mismatch**: Creates arbitrage when coupon period ≠ 182.5 days

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

from feature_engineering import DayCountCalculator

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

## 1. Day Count Calculation Examples

In [ ]:
# Example bond parameters
coupon_rate = 0.03  # 3% annual coupon
face_value = 100

# Different coupon period scenarios
scenarios = [
    {'name': '180 days', 'period_length': 180, 'days_accrued': 90},
    {'name': '181 days (TARGET)', 'period_length': 181, 'days_accrued': 90},
    {'name': '182 days (TARGET)', 'period_length': 182, 'days_accrued': 91},
    {'name': '183 days', 'period_length': 183, 'days_accrued': 91},
    {'name': '184 days', 'period_length': 184, 'days_accrued': 92},
]

results = []

for scenario in scenarios:
    period_length = scenario['period_length']
    days_accrued = scenario['days_accrued']
    
    # Calculate accrued interest using both conventions
    ai_actual_actual = DayCountCalculator.actual_actual(
        coupon_rate, days_accrued, period_length, face_value
    )
    
    ai_actual_365 = DayCountCalculator.actual_365(
        coupon_rate, days_accrued, face_value
    )
    
    # Calculate arbitrage profit
    arbitrage = DayCountCalculator.calculate_arbitrage_profit(
        coupon_rate, days_accrued, period_length, face_value
    )
    
    results.append({
        'Scenario': scenario['name'],
        'Period Length': period_length,
        'Days Accrued': days_accrued,
        'AI (Actual/Actual)': ai_actual_actual,
        'AI (Actual/365)': ai_actual_365,
        'Difference ($)': arbitrage,
        'Difference (bps)': (arbitrage / face_value) * 10000
    })

results_df = pd.DataFrame(results)
print("\nDay Count Convention Comparison")
print("=" * 100)
print(results_df.to_string(index=False))
print("=" * 100)

## 2. Visualize Arbitrage Opportunity

In [ ]:
# Plot the difference across different coupon period lengths
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Accrued interest comparison
x = results_df['Period Length']
axes[0].plot(x, results_df['AI (Actual/Actual)'], marker='o', label='Actual/Actual', linewidth=2)
axes[0].plot(x, results_df['AI (Actual/365)'], marker='s', label='Actual/365', linewidth=2)
axes[0].axvline(181, color='red', linestyle='--', alpha=0.5, label='Target Periods')
axes[0].axvline(182, color='red', linestyle='--', alpha=0.5)
axes[0].set_xlabel('Coupon Period Length (days)')
axes[0].set_ylabel('Accrued Interest ($)')
axes[0].set_title('Accrued Interest by Day Count Convention')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Plot 2: Arbitrage profit
colors = ['gray' if p not in [181, 182] else 'red' for p in results_df['Period Length']]
axes[1].bar(results_df['Scenario'], results_df['Difference (bps)'], color=colors, alpha=0.7)
axes[1].set_xlabel('Scenario')
axes[1].set_ylabel('Arbitrage Profit (bps)')
axes[1].set_title('Arbitrage Opportunity by Coupon Period')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n📊 Key Insight: The arbitrage is maximized when coupon period = 181 or 182 days!")

## 3. Sensitivity Analysis

In [ ]:
# Analyze how arbitrage profit changes with different parameters

# Vary coupon rate
coupon_rates = np.arange(0.01, 0.06, 0.005)  # 1% to 6%
period_length = 181  # Target period
days_accrued = 90

profits_by_coupon = []
for cr in coupon_rates:
    profit = DayCountCalculator.calculate_arbitrage_profit(
        cr, days_accrued, period_length, face_value
    )
    profits_by_coupon.append((profit / face_value) * 10000)  # Convert to bps

# Vary days accrued
days_accrued_range = range(1, period_length)
profits_by_days = []
for da in days_accrued_range:
    profit = DayCountCalculator.calculate_arbitrage_profit(
        0.03, da, period_length, face_value
    )
    profits_by_days.append((profit / face_value) * 10000)

# Plot sensitivity
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(coupon_rates * 100, profits_by_coupon, linewidth=2, color='blue')
axes[0].set_xlabel('Coupon Rate (%)')
axes[0].set_ylabel('Arbitrage Profit (bps)')
axes[0].set_title('Sensitivity to Coupon Rate (181-day period, 90 days accrued)')
axes[0].grid(True, alpha=0.3)

axes[1].plot(days_accrued_range, profits_by_days, linewidth=2, color='green')
axes[1].set_xlabel('Days Accrued')
axes[1].set_ylabel('Arbitrage Profit (bps)')
axes[1].set_title('Sensitivity to Days Accrued (181-day period, 3% coupon)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Real-World Example

In [ ]:
# Simulate a realistic trade scenario
print("\n" + "="*80)
print("REAL-WORLD TRADE EXAMPLE")
print("="*80)

# Bond parameters
bond_coupon = 0.0375  # 3.75% coupon
bond_price_clean = 98.50
period_length = 181
days_to_coupon = 3  # Buying 3 days before coupon
days_accrued = period_length - days_to_coupon

# Calculate arbitrage
arbitrage_per_100 = DayCountCalculator.calculate_arbitrage_profit(
    bond_coupon, days_accrued, period_length, 100
)
arbitrage_bps = (arbitrage_per_100 / bond_price_clean) * 10000

# Trade details
notional = 1_000_000  # $1M position
num_bonds = notional / bond_price_clean
total_arbitrage = arbitrage_per_100 * (num_bonds / 100)

# Costs
commission = notional * 0.0005  # 5 bps
slippage = notional * 0.0002  # 2 bps
total_costs = commission + slippage

# Net profit
net_profit = total_arbitrage - total_costs
net_return_bps = (net_profit / notional) * 10000

print(f"\nBond Details:")
print(f"  Coupon Rate: {bond_coupon*100:.2f}%")
print(f"  Clean Price: ${bond_price_clean:.2f}")
print(f"  Coupon Period: {period_length} days")
print(f"  Days to Coupon: {days_to_coupon}")

print(f"\nPosition:")
print(f"  Notional: ${notional:,.2f}")
print(f"  Number of Bonds: {num_bonds:,.2f}")

print(f"\nArbitrage Calculation:")
print(f"  Arbitrage per $100 face: ${arbitrage_per_100:.4f}")
print(f"  Arbitrage (bps): {arbitrage_bps:.2f}")
print(f"  Total Arbitrage Profit: ${total_arbitrage:,.2f}")

print(f"\nCosts:")
print(f"  Commission: ${commission:,.2f}")
print(f"  Slippage: ${slippage:,.2f}")
print(f"  Total Costs: ${total_costs:,.2f}")

print(f"\nNet Result:")
print(f"  Net Profit: ${net_profit:,.2f}")
print(f"  Net Return: {net_return_bps:.2f} bps")
print(f"  Holding Period: {days_to_coupon} days")
print(f"  Annualized Return: {(net_return_bps / days_to_coupon * 365):.2f} bps")

print("\n" + "="*80)

if net_profit > 0:
    print("✅ PROFITABLE ARBITRAGE")
else:
    print("❌ UNPROFITABLE - Costs exceed arbitrage profit")

print("="*80)

## 5. Key Takeaways

### Why This Works
1. **Structural Inefficiency**: Canadian bonds use different day count conventions for pricing vs settlement
2. **Maximum Impact**: Arbitrage is maximized when coupon period = 181 or 182 days
3. **Timing Critical**: Must enter just before coupon payment to capture the mispricing

### Risk Factors
1. **Transaction Costs**: Small profit margins can be erased by fees
2. **Execution Timing**: Even slight delays can eliminate profits
3. **182-Day Periods**: More sensitive due to being on the "edge" of the convention mismatch

### Strategy Requirements
1. Institutional-level transaction costs (< 5 bps)
2. Precise execution infrastructure
3. Duration hedging to isolate the day count effect
4. Monitoring for strategy crowding